In [1]:
val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._

val data = spark.read.
        option("inferSchema", true).
        option("header", false).
        csv("Data/KDD_Cup/kddcup.data_10_percent_corrected").
        toDF(
        "duration","protocol_type","service","flag",
        "src_bytes","dst_bytes","land","wrong_fragment","urgent",
        "hot","num_failed_logins","logged_in","num_compromised",
        "root_shell","su_attempted","num_root","num_file_creations",
        "num_shells","num_access_files","num_outbound_cmds","is_host_login",
        "is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
        "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
        "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
        "dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate",
        "dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate",
        "dst_host_srv_rerror_rate","label")

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1633842997123)
SparkSession available as 'spark'


sqlC: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@6ec312d1
import sqlC.implicits._
data: org.apache.spark.sql.DataFrame = [duration: int, protocol_type: string ... 40 more fields]


In [2]:
data.cache()

res0: data.type = [duration: int, protocol_type: string ... 40 more fields]


In [ ]:
data.select("label").groupBy("label").count().orderBy($"count".desc).show(25)

//23개의 레이블이 존재하고 스머프와 넵튠이 가장 많이 존재하는 것을 볼 수 있다.
//지금 이 레이블이 있지만 없다고 가정하고 k-mean 클러스트링을 진행한다.

In [ ]:
//앞에서 본 데이터 샘플을 보면 스트링 데이터 즉 여기선 카테고리컬 데이터가 있는 것을 알 수 있다.
//즉 숫자 데이터가 아닌 데이터가 들어있다는 것이다.
//이런 논 뉴머릭 피쳐들을 우리가 컨트롤 해야한다.
//즉 이런 숫자가 아닌 피쳐들도 우리가 사용해야 하니까 카테고리컬 한 피쳐들도 바꿔주어야 하는 것이다.
//k-민 클러스트링은 피쳐 도메인에서 센트로이드 포인트를 계산하고 그 센트로이드 포인트와의
//거리를 계산해야해서 뉴머릭 피쳐를 사용해야 한다.
//그래서 첫번째 클러스트링에서는 논 뉴머릭 밸류는 뺴고 뉴머릭 밸류들만 가지고 진행해보도록 한다.

val numericOnly = data.drop("protocol_type","service","flag").cache()
//drop을 이용해 논 뉴머릭 밸류들을 제외한 데이터 셋을 만든다.

In [5]:
import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.{OneHotEncoder, VectorAssembler, StringIndexer, StandardScaler}
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.sql.{DataFrame, SparkSession}
import scala.util.Random

import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.{OneHotEncoder, VectorAssembler, StringIndexer, StandardScaler}
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.sql.{DataFrame, SparkSession}
import scala.util.Random


In [ ]:
val assembler = new VectorAssembler().
    setInputCols(numericOnly.columns.filter(_ !="label")).
    setOutputCol("featuerVector")

val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")

val pipeline = new Pipeline().setStages(Array(assembler, kmeans))
val pipelineModel =  pipeline.fit(numericOnly)
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]

//벡터 어셈블러로 피쳐벡터 만들고 모델 만들어서 파이프라인으로 합쳐서 피팅을 하게 된다.
//필요한 라이브러리를 임포트 하고 피쳐벡터 만들고 k민 모델을 연결해서 피팅 할 것이다.

In [ ]:
//kmeansModel.clusterCenters.foreach(println)

//그냥 print로 보면 디맨션 수 만큼 숫자가 클러스터 마다 나열이 될 것이고 직관적으로
//이해하기는 어렵다. 그래도 알 수 있는 점은 각 클러스터가 어떤 센트로이드를 가지고 있는지 알 수 있다.

//만약 여기서 벡터가 2개가 나왔다면 k민즈 모델이 k=2로 피팅이 되었다는 이야기 이다.
//이걸 클러스터 2개로는 힘들었을 것이고 위는 예시이다.

In [ ]:
val withCluster = pipelineModel.transform(numericOnly)

withCluster.select("cluster","label").
    groupBy("cluster","label").count().
    orderBy($"cluster",$"count".desc).
    show(25)

//원래는 비지도 학습이라 정답 레이블이 없지만 우리는 예시를 보는 것이므로 레이블이 있다.
//그래서 실제로 클러스터가 잘 되었는지 결과를 확인하기 위해 레이블과 함꼐 확인 해보자.
//그런데 뉴머릭 데이터만 가지고 구분해보니까 딱 봐도 안된 걸 알 수 있다.
//즉 하나만 1이고 나머지는 다 0에 속하게 된 것을 알 수 있고 이걸 보고 뉴머릭만 쓰는 것은
//의미가 없구나를 알 수 있다.

In [ ]:
//k를 선택하는 것에 대해 엘보우 메소드를 적용한다.
//그러기 위해 컴퓨터코스트를 구해서 사용해야 한다.

def clusteringScore0(data: DataFrame, k: Int): Double = {
    val assembler = new VectorAssembler().
    setInputCols(data.columns.filter(_ !="label")).
    setOutputCol("featureVector")
    
    val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")
    
    val pipeline = new Pipeline().setStages(Array(assembler, kmeans))
    
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computerCost(assembler.transform(data)) / data.count()
}

//우리가 쉽게 정의할 수 있어서 이벨류에이터가 없다. 디시전 트리는 멀티클래스 클래시피케이션을 
//이벨류에이션 할 수 있는 에벨류에이터가 있어서 new 로 그냥 만들었었다.
//k민즈는 존재하지 않기 떄문에 그걸 만들어서 사용하겠다는 것이다.

//컴퓨터코스트 메소드는 코스트를 계산해주는 k민즈 모델에서 제공하는 멤버 함수이다.

In [ ]:
(20 to 100 by 20).map(k => (k, clusteringScore0(numericOnly, k))).foreach(println)

//그래서 우리가 만든 함수를 이용해서 서로 다른 k에 대해서 계산을 해볼 것이다.
//20 , 40 , 60 , 80 , 100 에 대해서 계산해서 확인해보자는 것이다.
//결과를 보면 k가 커지면서 코스트가 줄어는 것을 볼 수 있다.
//k가 80일 때 알고리즘이 서브 옵티멀에 스톱 했다 그리고 로컬 옵티멀에 도착하기 전에 반복이
//종료 되었다 라고 생각할 수 도 있다. 그런데 잘 보면 다시 커진 시점이 나온 것을 알 수 있고
//반복 즉 이터레이션 자체가 부족했다는 점을 알 수 있다.

In [ ]:
def clusteringScore1(data: DataFrame, k: Int): Double = {
    val assembler = new VectorAssembler().
    setInputCols(data.columns.filter(_ !="label")).
    setOutputCol("featureVector")
    
    val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")
    setMaxiter(40).
    setTol(1.0e-5)
    
    val pipeline = new Pipeline().setStages(Array(assembler, kmeans))
    
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computerCost(assembler.transform(data)) / data.count()
}

//Maxiter 즉 맥스 이터레이션의 디폴트는 20 으로 되어있고 , 디크리즈는 1.0e-4로 잡혀있다.
//아래쪽은 1의 10^-4승 이하로 움직이면 다 움직인 것으로 판단하겠다 라는 것이다.
//즉 센트로이드 포지션이 위쪽 보다 적게 움직이면 다 되었다 즉 컨벌즈 라고 판단하고
//위와 아래 둘중에 하나만 만족해도 빠져나가게 되는 것이다.
//그래서 맥스 이터레이션 수가 부족해서 이터레이션을 충분히 돌지 못하는 경우가 있다.

//위 함수를 사용해서 구해낸 코스트 값으로 그래프를 그릴려 하는데 값이 너무 크다.
//특정 피쳐는 레인지가 크고 또 어떤건 작고 하면 그 간격때문에 코스트에 영향을 미치는 것이
//너무 크다. 즉 1 ~ 1000 사이랑 2~5 사이가 있다면 1~1000 사이의 피쳐가 센트로이드에 영향을
//많이 미칠 것이다. 즉 영향이 다르다는 것이고 피쳐를 노멀라이제이션을 하는 것이 중요하다는 것이디.

//즉 머신러닝을 할 때는 모든 피쳐의 레인지가 같도록 노멀라이제이션을 하는 것이 좋다.
//노멀라이제이션 할 때 가우션 분포를 따를 수 있도록 피쳐에서 평균을 빼고 분산으로 나누어 주는
//이런 과정을 거친다. 스파크에 StandardScaler 가 있어 이런 과정을 파이프 라인에
//편하게 사용가능하다. 분산을 뺸다는 것은 스케일링 범위를 조정하는 것이고 뮤를 즉 평균을
//뺀다는 것은 중심점이 0에 오도록 평행이동 해주는 역할이다. 그래서 유클리드 관점에서는
//평균을 0에 맞추는 것은 별 상관이 없다. 그래서 간격만 보면 된다는 마인드로 뮤를 뺴는 것은
//뮤를 뺴는 것이 의미가 없다고 볼 수 있다. 그래서 뮤 안뺴고 나누기만 해도 된다는 마인드 인 것이다.

//그래서 위의 과정을 넣은 스코어 함수를 하나 더 만들도록 한다.

In [ ]:
def clusteringScore2(data: DataFrame, k: Int): Double = {
    val assembler = new VectorAssembler().
    setInputCols(data.columns.filter(_ !="label")).
    setOutputCol("featureVector")
    
    val scaler = new StandardScaler()
    .setInputCol("featureVector")
    .setOutputCol("scaledFeatureVector")
    .setWithStd(true)
    .setWithMean(false)
    
    val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")
    setMaxiter(40).
    setTol(1.0e-5)
    
    val pipeline = new Pipeline().setStages(Array(assembler, scaler ,kmeans))
    val pipelineModel = pipeline.fit(data)
    
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computerCost(assembler.transform(data)) / data.count()
}

//파이프 라인에 스케일러를 하나 넣어서 세가지 요소가 연결된 파이프 라인을 만든 것이다.

In [ ]:
(60 to 270 by 30).map(k => (k, clusteringScore2(numericOnly, k))).foreach(println)

//해보면 보기 좋은 값으로 표현되고 있음을 알 수 있다.
//스케일러를 넣기 떄문에 모든 피쳐를 공평하게 확인 할 수 있게 되었다.
//해보니 정확한 k를 찾기는 어려워 보인다. 왜 그렇냐면 카테고리컬 밸류인 스티링 밸류를
//제외시켰기 떄문이다.

//그래서 이 스트링 카테고리컬 밸류를 넘버릭으로 바꿔야 하는데 이때 사용할 수 있는 방법이
//원 핫 인코딩 이다. 아까 본 카테고리컬 밸류는 종류 중 하나만 가질 수 있다.
//그러면 3가지 후보중 하나기 떄문에 3차원 벡터로 표현하는 것이다.
//즉 1,0,0 / 0,1,0 / 0,0,1 이런 것이고 4개면 여기에 0 , 1 이 포함되도록 더해서 만들 것이다.
//왜 원 핫 인코더라고 하면 밸류들 중 하나만 핫 즉 1이기 떄문이다.

//그런데 숫자로 바꾸면 그냥 0 , 1 , 2 하면 안될까요? 하는데 그런 경우 문제는 
//각 카테고리 사이에 차이가 다르다는 점이다. 즉 0과 2 사이의 차이가 2인데 나머지는 1인 것이다.
//그러면 머신러닝은 어떻게 생각하냐면 2차이 나니까 1차이 나닌 카테고리들 끼리가 더 가까운
//카테고리 구나 한다는 것이다. 따라서 결과에 있어서 우리가 의도한 결과가 나오지 않을 수 있다는 것이다.

//그런 부분에 있어서 원 핫 인코더는 카테고리 사이사이의 거리가 모두 같게 되고 문제가 발생하지 않는다.

//위 과정을 하기 위해서는 일단 스트링인덱서로 0 1 2 이런식으로 맵핑한다.
//이후 맵핑된 수를 가지고 원 핫 인코더를 적용하면 된다.

In [ ]:
def oneHotPipeline(inputCols: String): (Pipeline, String) = {
    val indexer = new StringIndexer().
    setInputCol(inputCol).
    setOutputCol(inputCol + "_indexed")
    
    val encoder = new OneHotEncoder().
    setInputCol(inputCol + "_indexed").
    setOutputCol(inputCol + "_vec")
    
    val pipeline = new Pipeline().setStages(Array(indexer, encoder))
    (pipeline, inputCol + "_vec")
}

//이를 통해서 원 핫 인코더 파이프라인을 만든다.

In [ ]:
def clusteringScore3(data: DataFrame, k: Int): Double = {
    val (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    val (flagEncoder, flagVecCol) = oneHotPipeline("flag")
    
    val assembleCols = Set(data.columns: _*) --
    Seq("label","protocol_type","service","flag") ++
    Seq(protoTypeVecCol,serviceVecCol,flagVecCol)
    
    val assembler = new VectorAssembler().
    setInputCols(assembleCols.toArray).
    setOutputCol("featureVector")
    
    val scaler = new StandardScaler()
    .setInputCol("featureVector")
    .setOutputCol("scaledFeatureVector")
    .setWithStd(true)
    .setWithMean(false)
    
    val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")
    setMaxiter(40).
    setTol(1.0e-5)
    
    val pipeline = new Pipeline().setStages(
        Array((protoTypeEncoder,serviceEncoder,flagEncoder,assembler, scaler ,kmeans))
    val pipelineModel = pipeline.fit(data)
    
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computerCost(assembler.transform(data)) / data.count()
}

In [ ]:
(60 to 270 by 30).map(k => (k, clusteringScore3(numericOnly, k))).foreach(println)

//그래서 위의 경우로 확인해보니 명확한 엘보 포인트가 있는 데이터 였던 것을 알 수 있고
//이를 통해서 우리가 k를 120으로 두면 되겠구나를 판단 할 수 있다.
//그러면 이후 k를 120으로 고정해서 사용하면 된다.

In [ ]:
def fitPipeline4(data: DataFrame, k: Int): PipelineModel = {
    val (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    val (flagEncoder, flagVecCol) = oneHotPipeline("flag")
    
    val assembleCols = Set(data.columns: _*) --
    Seq("label","protocol_type","service","flag") ++
    Seq(protoTypeVecCol,serviceVecCol,flagVecCol)
    
    val assembler = new VectorAssembler().
    setInputCols(assembleCols.toArray).
    setOutputCol("featureVector")
    
    val scaler = new StandardScaler()
    .setInputCol("featureVector")
    .setOutputCol("scaledFeatureVector")
    .setWithStd(true)
    .setWithMean(false)
    
    val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")
    setMaxiter(40).
    setTol(1.0e-5)
    
    val pipeline = new Pipeline().setStages(
        Array((protoTypeEncoder,serviceEncoder,flagEncoder,assembler, scaler ,kmeans))
    val pipelineModel = pipeline.fit(data)
    
}
        
//k를 인풋으로 받을 수 있는 피팅 함수이다.

In [ ]:
val pipelineModel = fitPipeline4(data, 120)

//k를 120으로 두면 된다는 것을 알았기 떄문에 120으로 둔다.

val countByClusterLabel = pipelineModel.transform(data).
    select("cluster","label").
    groupBy("cluster","label").count().
    orderBy("Cluster","label")
countByClusterLabel.show()

//그래서 k를 120으로 고정해서 파이프라인을 만들어 트랜스폼 한 데이터에 대해서 확인을 하도록 한다.
//클러스터가 120개라서 다 보지는 못하지만 대략적인 부분을 확인 할 수 있다.

//anomaly detecter를 우리가 드디어 만들 수 있는 것이다.
//즉 새로운 데이터가 들어오면 센트로이드 120개와 비교해서 가까운 쪽으로 들어가기 되는 것이다.
//그런데 그 거리를 계산했는데 우리가 지정한 threshold 보다 크다면 분류 할 수 없는 새로운 데이터가
//들어온 것이므로 새로운 종류의 것이다 라고 이여기 한다는 것이다.

In [ ]:
val kMeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
val centroids = kMeansModel.clusterCenters

val clustered = pipelineModel.transform(data)
val threshold = clustered.
    select("cluster","scaledFeatureVector").as[(Int, Vector)].
    map { case (cluster, vec) => Vectors.sqdist(centroids(cluster),vec)}.
    orderBy($"value".desc).take(100).last

val originalCols = data.clumns
val anomalies = clustered.filter { row =>
    val cluster = row.getAs[Int]("cluster")
    val vec = row.getAs[Vector]("scaledFeatureVector")
    Vectors.sqdist(centroids(cluster),vec >= threshold)
}.select(originalCols.head, originalCols.tail:_*)

println(anomalies.first())

//anomaly detecter를 실제로 만들고 구현 시켜보는 코드에 대한 내용이다.